In [ ]:

import sys
sys.path.append("/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/mlr-xai")


from xai_selfies.WISP import *

In [ ]:
import sklearn
print(sklearn.__version__)

import rdkit
print(rdkit.__version__)

import shap
print(shap.__version__)

# To create Interative Parts

Interaktiv: \
Input als felder zum Text eingeben \
Model verfügbar? - Wo? - Feature function welche auf smiles arbeitet? \
Soll die explainability auch auf unbekannten daten arbeiten? - Test/Train split weg lassen?\



# DO WISP

In [ ]:
import shap
import inspect

def WISP(working_dir, input_dir, ID_Column_Name, Smiles_Column_Name, Target_Column_Name, model_available=False):
    '''
    If you have your model available please place it in the working_dir as model.pkl
    Input as comma seperated file
    '''
    #Interactive questions
    print('Do you already have a model you want to use for the evaluation?')
    if model_available is not None:
        print('Please provide the name of the function to create the features based on smiles as input:')
        function_name = input().strip()
        feature_function = globals()[function_name]

    #Load Data
    data = pd.read_csv(input_dir)
    data.rename(columns={ID_Column_Name: 'ID'}, inplace=True)

    #Preprocessing/Standadizing
    std = Standardizer(max_num_atoms=1000,#tipp jan: 100
                   max_num_tautomers=10,
                   include_stereoinfo=False,
                   keep_largest_fragment=True, 
                   canonicalize_tautomers=True, 
                   normalize=True, 
                   sanitize_mol=True)
    data["smiles_std"] = data[Smiles_Column_Name].apply(lambda smi: std(smi)[0]) 

    #save data in smi format
    data[['smiles_std', 'ID', Target_Column_Name]].to_csv(working_dir + "data.smi", sep='\t', index=False, header=False)

    if model_available is None:

        #Calculate Fingerprints as Descriptors
        data['Morgan_Fingerprint 2048Bit 2rad'] = data['smiles_std'].apply(get_morgan_fingerprint)
        data['MACCS_Fingerprint'] = data['smiles_std'].apply(get_MACCS_fingerprint)
        data['RDK_Fingerprint'] = data['smiles_std'].apply(get_RDK_fingerprint)

        #test/train split
        nr_test_samples = round(len(data) / 5) # 80/20 split
        test = data.sample(n=nr_test_samples, random_state=6)
        target_test = test[Target_Column_Name].values
        train = data.drop(test.index)
        plot_histogram(test, train,  Target_Column_Name, 'Exp.', 'Stucture Count', 'Test Set', 'Train Set',  working_dir + 'Count-Bins-test-train.png')

        #settings to find best model
        ALLfeatureCOLUMS = ['Morgan_Fingerprint 2048Bit 2rad',
                    'RDK_Fingerprint',
                    'MACCS_Fingerprint']
        
        model_types = [MLPRegressor(), 
               BayesianRidge(), 
               Lasso(), 
               GradientBoostingRegressor(), 
               LinearRegression(), 
               RandomForestRegressor(), 
               SVR(),
               GaussianProcessRegressor(kernel=Matern())]
        
        #find best model
        results = []

        for model_arc in model_types:          
            for feature in ALLfeatureCOLUMS:
                model, r2, R2, MAE, RMSE = hp_search_helper(model_arc,train,Target_Column_Name,[str(feature)])
                results.append({'Feature': feature,'Model_Type': model_arc,'Model': model,'r2': r2,'R2': R2,'MAE': MAE,'RMSE': RMSE})

        results_df = pd.DataFrame(results)
        best_model_row = results_df.loc[results_df['MAE'].idxmin()]
        model = best_model_row['Model']
        print('Best Model: ', best_model_row['Model'])
        print('With a MAE of: ', best_model_row['MAE'])
        print('Feature: ', best_model_row['Feature'])

        #pick feature function
        if best_model_row['Feature'] == 'Morgan_Fingerprint 2048Bit 2rad':
            feature_function = get_morgan_fingerprint
        if best_model_row['Feature'] == 'RDK_Fingerprint':
            feature_function = get_RDK_fingerprint
        if best_model_row['Feature'] == 'MACCS_Fingerprint':
            feature_function = get_MACCS_fingerprint

        #train model on trining set
        featureCOLUMS = [best_model_row['Feature']]

        prep_train = get_features(train, featureCOLUMS)
        target_train = train[Target_Column_Name].values
        model.fit(prep_train, target_train)

        #performance on test set
        prep_test = get_features(test, featureCOLUMS)
        predictions = model.predict(prep_test)

        #statistic on testset
        r2 = np.corrcoef(target_test.flatten(), predictions.flatten())[0,1]**2
        RMSE_z = np.sqrt(np.mean((target_test.flatten() - predictions.flatten())**2))
        RMSE_n = np.sqrt(np.mean((target_test.flatten() - np.mean(target_test.flatten()))**2))
        R2 = 1 - RMSE_z**2/RMSE_n**2
        MAE = mean_absolute_error(target_test.flatten(), predictions.flatten())
        max_err = max_error(target_test.flatten(), predictions.flatten())
        mse = mean_squared_error(target_test.flatten(), predictions.flatten())
        rmse = np.sqrt(mse)

        #print/plot results
        print('Performance on testset(r2, R2, MAE, RMSE, Maximal Error, MSE):',r2,';',R2,';',MAE,';',rmse,';',max_err,';', mse)

        r2 = np.corrcoef(predictions, target_test)[0,1]**2
        plot_2D(['r$^2$ = ' + str(f"{r2:.2f}")], 'upper left', predictions , target_test,
                'predicted', 'experimental', working_dir + '20-80-split-true-pred.png', '#A43341', 
                include_line=False, line_style='None')
        
        #save model training results
        results_df.to_csv(working_dir + "Grid-Search.csv", index=False)
        pickle.dump(model, open(working_dir + "model.pkl", "wb"))
    
    #load the provided or just trained model
    model = pickle.load(open(working_dir + "model.pkl", 'rb'))

    #Attribute Atoms
    Attribution_Colums = ['Atom Attributions']#, 'Path Attributions', 'Dropout Attributions'
    color_coding =['#10384f'] #,'#89d329','#00bcff'
    
    #model/descriptor agnostic
    data['Atom Attributions'] = data['smiles_std'].apply(lambda s: attribute_atoms(s, model, feature_function))
    #data['Dropout Attributions'] = data['smiles_std'].apply(lambda s: attribute_atoms_dropout(s, model, feature_function))
    #data['Path Attributions'] = data['smiles_std'].apply(lambda s: attribute_atoms_paths(s, model, feature_function))
    
    #SHAP explainer
    if "Morgan" in inspect.getsource(feature_function):#to only use on Morgan
        data['Morgan_Fingerprint 2048Bit 2rad'] = data['smiles_std'].apply(feature_function)
        #pick explainer
        model_type = model.named_steps['model'].__class__.__name__
        if model_type in ['GradientBoostingRegressor', 'RandomForestRegressor']:
            explainer = shap.TreeExplainer(model.named_steps['model'])
        if model_type in ['MLPRegressor', 'SVR', 'GaussianProcessRegressor'] :
            prep_data = get_features(data, ['Morgan_Fingerprint 2048Bit 2rad'])
            explainer = shap.KernelExplainer(model.predict, model.named_steps['scaler'].transform(prep_data))
        if model_type in ['BayesianRidge', 'Lasso', 'LinearRegression'] :
            prep_data = get_features(data, ['Morgan_Fingerprint 2048Bit 2rad'])
            explainer = shap.LinearExplainer(model.named_steps['model'],model.named_steps['scaler'].transform(prep_data))
        

        data = get_SHAP_Morgan_attributions(data, 'Morgan_Fingerprint 2048Bit 2rad', 'smiles_std', model, explainer)
        
        Attribution_Colums.append('SHAP Attributions')
        color_coding.append('#9C0D38')
    
    #RDKit
    if "Morgan" in inspect.getsource(feature_function):
        data['RDKit Attributions'] = data['smiles_std'].apply(lambda s: RDKit_attributor(s, SimilarityMaps.GetMorganFingerprint, model))
        Attribution_Colums.append('RDKit Attributions')
        color_coding.append('#758ECD')
    if "RDK" in inspect.getsource(feature_function):
        def fp_func(m, a):
            return SimilarityMaps.GetRDKFingerprint(m, atomId=a, maxPath=7)
        data['RDKit Attributions'] = data['smiles_std'].apply(lambda s: RDKit_attributor(s, fp_func, model))
        Attribution_Colums.append('RDKit Attributions')
        color_coding.append('#758ECD')

    #Creat MMP database
    colums_to_keep = Attribution_Colums + [Target_Column_Name]
    data_MMPs = create_MMP_database(working_dir + "data.smi", working_dir ,data, colums_to_keep)
    data_MMPs.to_csv(working_dir + "MMPs_with_attributions.csv", index=False)

    #add predictions

    #for 1
    data_MMPs['Feature_1'] = data_MMPs['smiles_1'].apply(feature_function)
    X_data_attributions_1 = get_features(data_MMPs, ['Feature_1'])
    predictions_1 = model.predict(X_data_attributions_1)
    data_MMPs['predictions_1'] = predictions_1

    #for 2
    data_MMPs['Feature_2'] = data_MMPs['smiles_2'].apply(feature_function)
    X_data_attributions_1 = get_features(data_MMPs, ['Feature_2'])
    predictions_1 = model.predict(X_data_attributions_1)
    data_MMPs['predictions_2'] = predictions_1

    #Add indices
    data_MMPs[["unmatched_atom_index_1", "unmatched_atom_index_2"]] = data_MMPs.apply(
    lambda row: pd.Series(get_unmatched_atom_indices_fragments(row["smiles_1"], row["smiles_2"], row["constant"])), axis=1)

    #Add Plots

    for attr_method, color in zip(Attribution_Colums, color_coding):
        
        r2_whole, r2_fragment, data_MMPs = get_r2_and_summed_data_attributions(data_MMPs, 'predictions_1', 'predictions_2', attr_method + '_1', attr_method + '_2', attr_method, 'unmatched_atom_index_1' , 'unmatched_atom_index_2')

        plot_2D(['$r^2$(' + attr_method + ') = ' + str(f"{r2_whole:.2f}")], 'upper left', data_MMPs['delta_prediction'], data_MMPs['delta_sum_' + attr_method],
                '$\Delta$Predictions MMP', '$\Delta$Attributions MMP (whole Mol)', working_dir + 'PREDvsCONTRIBUTIONSwhole' + attr_method + '.png', 
                color,
                include_line=True, line_style='None')

        plot_2D(['$r^2$(' + attr_method + ') = ' + str(f"{r2_fragment:.2f}")], 'upper left', data_MMPs['delta_prediction'], data_MMPs['delta_sum_fragment_contributions_' + attr_method],
                '$\Delta$Predictions MMP', '$\Delta$Attributions MMP (Fragment)', working_dir + 'PREDvsCONTRIBUTIONSfragment' + attr_method + '.png', 
                color,
                include_line=True, line_style='None')


        data_MMPs['delta_target'] = data_MMPs[Target_Column_Name + '_1'] - data_MMPs[Target_Column_Name + '_2']

        r2 = np.corrcoef(data_MMPs['delta_target'], data_MMPs['delta_sum_' + attr_method])[0,1]**2

        plot_2D(['$r^2$(' + attr_method + ') = ' + str(f"{r2:.2f}")], 'upper left', data_MMPs['delta_target'], data_MMPs['delta_sum_' + attr_method],
                '$\Delta$Target MMP', '$\Delta$Attributions MMP (whole Mol)', working_dir + 'EXPvsCONTRIBUTIONSwhole' + attr_method + '.png', 
                color,
                include_line=True, line_style='None')
        
    #Analyse Locality

    data_MMPs['unmatched_atom_index_1_with_neighbors'] = data_MMPs.apply(lambda row: get_neighbors(row['smiles_1'], row['unmatched_atom_index_1']), axis=1)
    data_MMPs['unmatched_atom_index_2_with_neighbors'] = data_MMPs.apply(lambda row: get_neighbors(row['smiles_2'], row['unmatched_atom_index_2']), axis=1)

    methods = {
    'Atom': ('Atom Attributions_1_fix', 'Atom Attributions_2_fix', '#10384f', 'ATOM')}

    if 'SHAP Attributions' in Attribution_Colums:
        methods['SHAP'] = ('SHAP Attributions_1_fix', 'SHAP Attributions_2_fix', '#9C0D38', 'SHAP')
    if 'RDKit Attributions' in Attribution_Colums:
        methods['RDKit'] = ('RDKit Attributions_1_fix', 'RDKit Attributions_2_fix', '#758ECD', 'RDKIT')

    for method, (attr1, attr2, color, label) in methods.items():
        key1 = f'summ_unmatched_{method}_contributions_1_sphere'
        key2 = f'summ_unmatched_{method}_contributions_2_sphere'
        delta_key = f'delta_sum_fragment_{method}_contributions_sphere'

        data_MMPs[key1] = get_unmatched_attributions(data_MMPs, attr1, 'unmatched_atom_index_1_with_neighbors')
        data_MMPs[key2] = get_unmatched_attributions(data_MMPs, attr2, 'unmatched_atom_index_2_with_neighbors')
        data_MMPs[delta_key] = data_MMPs[key1] - data_MMPs[key2]

        r2 = np.corrcoef(data_MMPs['delta_prediction'], data_MMPs[delta_key])[0,1]**2

        plot_2D(
            [f'$r^2$({method}) = {r2:.2f}'], 'upper left',
            data_MMPs['delta_prediction'],
            data_MMPs[delta_key],
            '$\Delta$Predictions MMP',
            '$\Delta$Attributions MMP (Fragment+Neig)',
            working_dir + label + 'attributor_NEIG.png',
            color,
            include_line=True,
            line_style='None'
        )
    
    #test/train dependency   
    if model_available is None:
        data_MMPs['set_1'] = data_MMPs['smiles_1'].isin(test['smiles_std']).map({True: 'test', False: 'train'})
        data_MMPs['set_2'] = data_MMPs['smiles_2'].isin(test['smiles_std']).map({True: 'test', False: 'train'})

        def compute_r2(data, label):
            mask = data['set_1'].str.contains(label) & data['set_2'].str.contains(label)
            filtered = data[mask]
            for attr in Attribution_Colums:
                r2 = np.corrcoef(
                    filtered['delta_prediction'],
                    filtered[f'delta_sum_{attr}']
                )[0, 1] ** 2
                print(f'{attr}_{label}:', r2)
        
        for split in ['train', 'test']:
            compute_r2(data_MMPs, split)
    
    data_MMPs.to_csv(working_dir + "Complete_Data.csv", index=False)

    return data_MMPs

## Crippen logP ZINC (with ML)

In [ ]:
'''
data = WISP('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/Crippen-Rassmussen/', 
     '/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/Crippen-Rassmussen/ZINC-Crippen.csv',
     'No',
     "smiles", 
     'crippen_logP',
     model_available=None)
'''

## Crippen logP MolNet (with ML)

In [ ]:
data = WISP('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/Crippen-MolNet/', 
     '/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/Crippen-MolNet/Lipophilicity-Crippen.csv',
     'CMPD_CHEMBLID',
     "smiles", 
     'crippen_logP',
     model_available=None)

## LogP (Morgan + Random Forest Model)

In [ ]:
data = WISP('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/Lipophilicity/', 
     '/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/Lipophilicity/Lipophilicity.csv',
     'CMPD_CHEMBLID',
     "smiles", 
     'exp',
     model_available=True)

## PIXIE

In [ ]:
data = WISP('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/PIXIE/', 
     '/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/PIXIE/data_complete_descriptors.txt',
     'Structure Name',
     "SMILES Product", 
     'LCAP Scheme 25',
     model_available=True)

## Solubility

In [ ]:
data = WISP('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/ESOLmoleculeNET/', 
     '/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/ESOLmoleculeNET/delaney-processed-Nr.csv',
     "Nr",
     "smiles", 
     'measured log solubility in mols per litre',
     model_available=None)

## TRPV1 bioactivity

In [ ]:
data = WISP('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/TRPV1/', 
     '/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/TRPV1/TRPV1-preprocessed-molpipeline-ChEMBL34.csv',
     'Molecule ChEMBL ID',
     "Smiles", 
     'logaritmic_activity',
     model_available=None)

## Factor Xa

In [ ]:
data = WISP('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/FactorXa-Bail2023/', 
     '/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/FactorXa-Bail2023/FXa_Final_Public_Analogs_v4.csv',
     'Structure No',
     "Smiles", 
     'pKi',
     model_available=None)

## AMES Mutagenicity

In [ ]:
data = WISP('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/AMESwinter/', 
     '/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/AMESwinter/ames.csv',
     'Nr',
     "smiles", 
     'label',
     model_available=None)

# Analysis

#### Draw Molecules with Attributions

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import rdMolDraw2D
from IPython.display import Image

def draw_mol_with_attr(smiles, attr):
    # Create a molecule (Ethanol)
    mol = Chem.MolFromSmiles(smiles)

    # Example array of values (e.g., from calculations or descriptors)
    values = attr  # one value per atom

    drawer = rdMolDraw2D.MolDraw2DCairo(300, 300)

    # Access the drawing options
    options = drawer.drawOptions()

    # Assign labels to specific atoms
    for idx, value in enumerate(values):
        options.atomLabels[idx] = f'{value:.2f}'

    # Prepare and draw the molecule
    rdMolDraw2D.PrepareAndDrawMolecule(drawer, mol)
    drawer.FinishDrawing()

    # Display in Jupyter Notebook
    return Image(data=drawer.GetDrawingText())

In [ ]:
attr = [-0.08457939393939216, -0.11288095238094947, -0.09256190476190243, -0.13132476190475864, -0.08968412698412474, -0.1224875315840593, -0.24861746355684605, -0.22827270165208463, -0.3312164852607648, 0, -0.2023160544217648, -0.2023160544217648, -0.5672063259338717, -0.5382852805823986, -0.2781543876357507, -3.451325509177458, -3.3954510164551635, -3.1463015821594524, -3.116460049056739, -0.5999543250984525, -0.33972487852283145, -0.2782892857142778, -0.059171428571422366, -0.06663333333332677, -0.06109714285713488, -0.10144285714284962, -0.16538277056276424, -0.11197285714285025, -0.5424763964673487, -0.14360671525752874, -0.05259761904761706, -0.15345693877550762, -0.7976817460317353, -0.05821938775510005, -0.16587632653060944]
draw_mol_with_attr('Cc1cn(C2CCCN(S(=O)(=O)c3ccc(C(=O)O)c(Oc4cccc(Cl)c4)c3)C2)c(=O)[nH]c1=O', attr)

In [ ]:
attr = [-0.09669318181818208, -0.06688571428571463, -0.06145918367346963, -0.17660816326530654, -1.4515969620761084, -0.3008983432356228, -1.3369199865743835, -1.255931053024057, -1.3111168009536645, -1.460849684552024, -1.2496940391150446, -1.2163843488053536, -0.051957142857142924, -0.08889285714285744, -0.07538163265306132, -0.08704285714285698, -0.023373469387754833, -0.023373469387754895, -0.0805857142857146, -0.05498000000000016, -0.02898000000000034, -0.053114285714286114, -0.019050000000000372, -0.00872857142857153, -0.06918571428571452, -0.0419257142857147, -1.1732615079565476, -1.2053348531860308, 0.04665714285714235, -3.481891516224604, -3.41429544143642, -3.412031304240653, -3.2563618702445747]
draw_mol_with_attr('CC(=O)Nc1ccc2c(c1)c(-c1cc(NC3CC3)n3ncc(C#N)c3n1)cn2CCC(=O)O', attr)

In [ ]:
attr = [-0.3680934632034507, -0.6355469727891109, -0.9828836507936453, -0.6355469727891109, -0.3680934632034507, 1.28833698639456, 2.5815866233766194]
draw_mol_with_attr('CCC(CC)CO', attr)

In [ ]:
attr = [-0.6840650649350497, -1.5161986224489716, -2.521121428571412, -1.1637387012986953, -1.5161986224489716, -0.6840650649350497]
draw_mol_with_attr('CCC(C)CC', attr)

#### Draw Mol with Indices

In [ ]:
smiles_list = ['CC(C)(C)NC(=O)c1ncc(-c2ccc(C#N)cc2)o1', 'COC(=O)c1ccc(-c2cnc(C(=O)NC(C)(C)C)o2)cc1']
for smi in smiles_list:
    mol1 = Chem.MolFromSmiles(smi, sanitize=False)
    Chem.SanitizeMol(mol1)#to keep the explicit hydrogens
    for atom in mol1.GetAtoms():
        atom.SetProp('molAtomMapNumber', str(atom.GetIdx()))
        
    img = Draw.MolToImage(mol1)
    display(img)

In [ ]:
for smi in smiles_list:
    mol = Chem.MolFromSmiles(smi, sanitize=False)
    Chem.SanitizeMol(mol)
    
    # Set atom map numbers just for visualization (optional)
    for atom in mol.GetAtoms():
        atom.SetProp('molAtomMapNumber', str(atom.GetIdx()))
    
    # Get and print bond indices
    print(f"Bonds in: {smi}")
    for bond in mol.GetBonds():
        bond_idx = bond.GetIdx()
        begin_idx = bond.GetBeginAtomIdx()
        end_idx = bond.GetEndAtomIdx()
        print(f"Bond {bond_idx}: {begin_idx} - {end_idx}")
    
    img = Draw.MolToImage(mol)
    display(img)

#### Draw Attributions

In [ ]:
# For a single molecule

from rdkit.Chem import Draw
from rdkit.Chem.Draw import SimilarityMaps
from IPython.display import Image
from matplotlib.colors import LinearSegmentedColormap

smiles_list = ['CC(=O)OCN1C(=O)NC(c2ccccc2)(c2ccccc2)C1=O'
               , 'CCCCCC(=O)OCN1C(=O)NC(c2ccccc2)(c2ccccc2)C1=O'
                ]
attr_list = [[0.0878029485291862, 0.18429023419510093, 0.004651627523322409, 0.0993537583585071, 0.16879923401475366, 0.36112685890584145, 0.47997814021490615, 0.5133176109505179, -0.1555064997665855, -0.5548520776948712, -0.025468312563339257, 0.37974263117933693, 0.2491675745533543, 0.09273801343718749, 0.2727059394215573, 0.1264606792172728, -0.025468312563339257, 0.37974263117933693, 0.2491675745533543, 0.09273801343718749, 0.2727059394215573, 0.1264606792172728, -0.11780881485086714, 0.6290783248481869]
             , [-0.9578153449023629, -0.8851993337494158, -0.9503404452148967, -0.9969940568402514, -0.9524425781553383, -0.893346157518293, -0.45246240655922165, -0.6186790708403221, -0.7088811451528174, -0.669886992136049, -0.4721223080105039, 0.011893344695896932, -0.880420581582671, -1.2400228850140607, -0.80645822200806, -0.21699881390959047, -0.20136908485470856, -0.15630011556825388, -0.19265847840426514, -0.393178529021451, -0.80645822200806, -0.21699881390959047, -0.20136908485470856, -0.15630011556825388, -0.19265847840426514, -0.393178529021451, -0.35213751146935146, -0.09998442615988257]
             ]

for smiles, attributions in zip(smiles_list, attr_list):

    mol = Chem.MolFromSmiles(smiles, sanitize=False)
    Chem.SanitizeMol(mol)#to keep the explicit hydrogens

    custom_colors = ['#10384f', '#ffffff', '#9C0D38']#'#9C0D38', '#ffffff', '#10384f' oder (TU/Wien) #A43341', '#ffffff', '#3c629f'
    custom_cmap = LinearSegmentedColormap.from_list("custom", custom_colors, N=256)

    # Draw similarity map
    d = Draw.MolDraw2DCairo(300, 300)
    SimilarityMaps.GetSimilarityMapFromWeights(mol, attributions, draw2d=d, colorMap=custom_cmap)
    d.FinishDrawing()

    png = d.GetDrawingText()
    display(Image(data=png))

In [ ]:
print('index nitrogen:', attr_list[0][16])
print('index methoxy:', attr_list[1][0], attr_list[1][1])

#### Bit collisions?

In [ ]:
m = Chem.MolFromSmiles('CC(C)(C)NC(=O)c1ncc(-c2ccc(C#N)cc2)o1')
bit_info = {}
fp_1 = Chem.RDKFingerprint(m, maxPath=7, bitInfo=bit_info)

keys_with_15 = [key for key, paths in bit_info.items() if any(15 in path for path in paths)]
print(keys_with_15)


In [ ]:
m = Chem.MolFromSmiles('COC(=O)c1ccc(-c2cnc(C(=O)NC(C)(C)C)o2)cc1')
bit_info = {}
fp_2 = Chem.RDKFingerprint(m, maxPath=7, bitInfo=bit_info)

keys_with_0 = [key for key, paths in bit_info.items() if any(0 in path for path in paths)]
print(keys_with_0)

keys_with_1 = [key for key, paths in bit_info.items() if any(1 in path for path in paths)]
print(keys_with_1)

In [ ]:
common_elements = list(set(keys_with_15) & set(keys_with_1))
print(common_elements)

In [ ]:
for i in range(21):
    m = Chem.MolFromSmiles('CC(C)(C)NC(=O)c1ncc(-c2ccc(C#N)cc2)o1')
    bit_info = {}
    fp_1 = Chem.RDKFingerprint(m, maxPath=7, bitInfo=bit_info)

    keys_with_15 = [key for key, paths in bit_info.items() if any(i in path for path in paths)]
    print(i, len(keys_with_15
                 ))

In [ ]:
for i in range(23):
    m = Chem.MolFromSmiles('COC(=O)c1ccc(-c2cnc(C(=O)NC(C)(C)C)o2)cc1')
    bit_info = {}
    fp_1 = Chem.RDKFingerprint(m, maxPath=7, bitInfo=bit_info)

    keys_with_15 = [key for key, paths in bit_info.items() if any(i in path for path in paths)]
    print(i, len(keys_with_15
                 ))

#### What about the PIXIE atom attributions?

In [ ]:
smiles = "COC(=O)c1ccc(-c2cnc(C(=O)NC(C)(C)C)o2)cc1" 
mutated_dict = {}

for atom_idx, old_sym, new_sym, mutated in mutate_atoms(smiles):
    if atom_idx not in mutated_dict:
        mutated_dict[atom_idx] = []
    if mutated:
        mutated_dict[atom_idx].append(mutated)

    print(f"Atom {atom_idx} ({old_sym} → {new_sym}): {mutated}")

#does the number of attributions have something to do with that? --> no
#for i in mutated_dict:
    #print('index:', i, '; no mutations', len(mutated_dict[i]))

'''
#is it a specific mutation?
model = model = pickle.load(open("/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/PIXIE/model.pkl", 'rb'))
pred_origi = predictor_on_smiles(smiles, get_RDK_fingerprint, model)
print(pred_origi)

#müssen tiefe pred sein bei C (0)
mut_H = '[H]OC(=O)c1ccc(-c2cnc(C(=O)NC(C)(C)C)o2)cc1'
mut_B= 'BOC(=O)c1ccc(-c2cnc(C(=O)NC(C)(C)C)o2)cc1'
mut_N = 'CC(C)(C)NC(=O)c1ncc(-c2ccc(C(=O)ON)cc2)o1'
mut_O = 'CC(C)(C)NC(=O)c1ncc(-c2ccc(C(=O)OO)cc2)o1'
mut_F= 'CC(C)(C)NC(=O)c1ncc(-c2ccc(C(=O)OF)cc2)o1'
mut_si= 'CC(C)(C)NC(=O)c1ncc(-c2ccc(C(=O)O[SiH3])cc2)o1'
mut_P= 'CC(C)(C)NC(=O)c1ncc(-c2ccc(C(=O)OP)cc2)o1'
mut_S= 'CC(C)(C)NC(=O)c1ncc(-c2ccc(C(=O)OS)cc2)o1'
mut_Cl = 'CC(C)(C)NC(=O)c1ncc(-c2ccc(C(=O)OCl)cc2)o1'
mut_Br = 'CC(C)(C)NC(=O)c1ncc(-c2ccc(C(=O)OBr)cc2)o1'
mut_I= 'CC(C)(C)NC(=O)c1ncc(-c2ccc(C(=O)OI)cc2)o1'
print("mutations")
#pred_mut_H =
pred_mut_H = predictor_on_smiles(mut_H, get_RDK_fingerprint, model)
pred_mut_N = predictor_on_smiles(mut_N, get_RDK_fingerprint, model)
pred_mut_si = predictor_on_smiles(mut_si, get_RDK_fingerprint, model)
pred_mut_I = predictor_on_smiles(mut_I, get_RDK_fingerprint, model)
pred_mut_S = predictor_on_smiles(mut_S, get_RDK_fingerprint, model)
pred_mut_Cl = predictor_on_smiles(mut_Cl, get_RDK_fingerprint, model)
print(pred_mut_H, pred_mut_I, pred_mut_si, pred_mut_Cl, pred_mut_S, pred_mut_I)
#si, S
'''


In [ ]:
smiles = "COC(=O)c1ccc(-c2cnc(C(=O)NC(C)(C)C)o2)cc1" 
mutated_dict = {}

for atom_idx, old_sym, new_sym, mutated in mutate_atoms(smiles):
    if atom_idx not in mutated_dict:
        mutated_dict[atom_idx] = []
    if mutated:
        mutated_dict[atom_idx].append(mutated)



#### Draw Molecules (Heatmap)

In [ ]:
from rdkit.Chem import Draw
from rdkit.Chem.Draw import SimilarityMaps
from IPython.display import Image
from matplotlib.colors import LinearSegmentedColormap
import os
import ast

def gen_heatmap(data, index, output_dir, smiles_column, attribution_column, ID_column):
    '''
    Red is positive while blue is negative (to be consistant with PIXIE)
    '''
    smiles = data[smiles_column][index]
    attributions = ast.literal_eval(data[attribution_column][index])
    mol_id = data[ID_column][index]

    mol = Chem.MolFromSmiles(smiles, sanitize=False)
    Chem.SanitizeMol(mol)#to keep the explicit hydrogens

    custom_colors = ['#10384f', '#ffffff', '#9C0D38']#'#9C0D38', '#ffffff', '#10384f' oder (TU/Wien) #A43341', '#ffffff', '#3c629f'
    custom_cmap = LinearSegmentedColormap.from_list("custom", custom_colors, N=256)

    # Draw similarity map
    d = Draw.MolDraw2DCairo(900, 900)
    SimilarityMaps.GetSimilarityMapFromWeights(mol, attributions, draw2d=d, colorMap=custom_cmap)
    d.FinishDrawing()

    # Define and write output path
    filename = f"{mol_id}_{attribution_column}.png"
    output_path = os.path.join(output_dir, filename)
    with open(output_path, "wb") as f:
        f.write(d.GetDrawingText())

    return Image(filename=output_path)

In [ ]:
data_LCAP = pd.read_csv('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/Lipophilicity/Complete_Data.csv')
data_LCAP['Atom Attributions_2_fix']
#max_abs_value = data_LCAP['Atom Attributions_1_fix'].apply(lambda lst: max(map(abs, lst))).max()
#print(max_abs_value)

evaluated_series = data_LCAP['Atom Attributions_1_fix'].apply(ast.literal_eval)

# Find max absolute value
max_abs_value = evaluated_series.apply(lambda lst: max(map(abs, lst))).max()
print(max_abs_value)

In [ ]:
data_LCAP = pd.read_csv('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/Lipophilicity/Complete_Data.csv')
for index, row in data_LCAP.iterrows():
    gen_heatmap(
        data_LCAP,
        index,
        '/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/Lipophilicity/HeatMaps/',
        'smiles_2',
        'SHAP Attributions_2_fix',
        'ID_2'
    )

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

# Define the custom colormap
colors = ['#10384f', '#ffffff', '#9C0D38']
cmap = LinearSegmentedColormap.from_list("custom_diverging", colors)

# Create the colorbar
gradient = np.linspace(0, 1, 256).reshape(1, -1)
plt.imshow(gradient, aspect='auto', cmap=cmap)
plt.axis('off')
plt.show()

In [ ]:
from rdkit.Chem import Draw
from IPython.display import Image
from matplotlib.colors import LinearSegmentedColormap
import os
import ast
from matplotlib.colors import TwoSlopeNorm, LinearSegmentedColormap
from matplotlib.colorbar import ColorbarBase
import matplotlib.pyplot as plt
import os
from rdkit import Chem


def generate_heatmap(data, index, output_dir, smiles_column, attribution_column, ID_column, task_type):
    '''
    Red is positive while blue is negative (to be consistant with PIXIE)
    '''
    smiles = data[smiles_column][index]
    attributions = data[attribution_column][index]#.fillna(0)#ast.literal_eval()
    attributions = [0 if isinstance(x, float) and math.isnan(x) else x for x in attributions]
    mol_id = data[ID_column][index]

    #vmax = data[attribution_column].abs().max() * 0.7
    #evaluated_series = data[attribution_column].apply(ast.literal_eval)
    if task_type == 'regression':
        vmax = data[attribution_column].apply(lambda lst: max(map(abs, lst))).max() * 0.7
    if task_type == 'classification':
        vmax = 0.7
    #evaluated_series = data[attribution_column]
    #vmax = evaluated_series.apply(lambda lst: max(map(abs, list(lst)))).max() * 0.7
    #vmax = evaluated_series.apply(lambda lst: max(map(abs, lst))).max() * 0.7
    vmin = -vmax

    mol = Chem.MolFromSmiles(smiles, sanitize=False)
    Chem.SanitizeMol(mol)#to keep the explicit hydrogens

    # Draw similarity map
    atom_colors = {}
    for atom in mol.GetAtoms():
        idx = atom.GetIdx()
        weight = attributions[idx] if idx < len(attributions) else 0.0  # Default weight as 0.0 if atom index not found
        color, cmap = get_color(weight, '#10384f', '#ffffff', '#9C0D38', vmin, vmax)
        atom_colors[idx] = color
        atom.SetProp("_displayColor", ','.join(map(str, color)))
    
    d = Draw.MolDraw2DCairo(1800, 1800)
    d.DrawMolecule(mol,highlightAtoms=list(atom_colors.keys()),highlightAtomColors=atom_colors)#, highlightBonds=[]
    d.FinishDrawing()

    # Create a color bar
    fig, ax = plt.subplots(figsize=(6, 1))
    fig.subplots_adjust(bottom=0.5)

    norm = TwoSlopeNorm(vmin=vmin, vcenter=0.0, vmax=vmax)
    cbar = ColorbarBase(ax, norm=norm, cmap=cmap, orientation='horizontal')  #

    tick_positions = [vmin, vmax]
    tick_labels = [vmin, vmax]
    cbar.set_ticks(tick_positions)
    cbar.set_ticklabels(tick_labels)

    plt.savefig(os.path.join(output_dir, 'Legend.png'), dpi=300, bbox_inches='tight')
    plt.close()

    # Define and write output path
    filename = f"{mol_id}_{attribution_column}.png"
    output_path = os.path.join(output_dir, filename)
    with open(output_path, "wb") as f:
        f.write(d.GetDrawingText())

    return Image(filename=output_path)

def get_color(weight, neg_color, neut_color, pos_color, vmin, vmax):
    """
    Creates a custom colormap from black to red and colors the bonds accordingly.

    Keyword arguments:
    -- weight: Weight of the bond to be colored.
    -- neg_color: Color with which the decrease of the target variable should be described.
    -- neut_color: Color with which no influence of the target variable should be described.
    -- pos_color: Color with which the increase of the target variable should be described.
    -- vmin: Lower bound to normalize the colourcoding on the bond weights.
    -- vmax: Upper bound to normalize the colourcoding on the bond weights.

    Returns:
    -- tuple(rgba[:3]): Colour of the bond.
    -- cmap: Custom color map.
    """
    norm = TwoSlopeNorm(vmin=vmin, vcenter=0.0, vmax=vmax)
    colors = [neg_color, neut_color, pos_color] # 'dimgray'
    cmap = LinearSegmentedColormap.from_list("custom_cmap", colors)
    rgba = cmap(norm(weight))
    return tuple(rgba[:3]), cmap


In [ ]:
data_LCAP = pd.read_csv('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/Lipophilicity/Complete_Data.csv')
for index, row in data_LCAP.iterrows():
    generate_heatmap(
        data_LCAP,
        index,
        '/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/Lipophilicity/HeatMaps/',
        'smiles_2',
        'Atom Attributions_2_fix',
        'ID_2',
        'classification'
    )

#### Correlation Analysis

In [ ]:
import pandas as pd
from scipy.stats import zscore

def correlation_contrib(data, contrib_colum):
    # Standardize both columns
    data['z_pred'] = zscore(data['delta_prediction'])
    data['z_attr'] = zscore(data[contrib_colum])

    # Pointwise contribution to Pearson correlation
    data['correlation_contrib' + contrib_colum] = data['z_pred'] * data['z_attr']
    return data

    # Best (most positively contributing) datapoints
    #most_positively_correlated = data.nlargest(10, 'correlation_contrib')

    # Worst (most negatively contributing) datapoints
    #most_negatively_correlated = data.nsmallest(10, 'correlation_contrib')

    # Show results
    #print("Most positively correlated datapoints:")
    #print(most_positively_correlated[['delta_prediction', 'delta_sum_fragment_contributions_Atom Attributions', 'correlation_contrib']])
    #print(most_positively_correlated[['ID_1', 'ID_2']])

    #print("\nMost negatively correlated datapoints:")
    #print(most_negatively_correlated[['delta_prediction', 'delta_sum_fragment_contributions_Atom Attributions', 'correlation_contrib']])
    #print(most_negatively_correlated[['ID_1', 'ID_2']])


def get_residuals(data, contrib_colum):
    X = data[['delta_prediction']].values.reshape(-1, 1)
    Y = data[contrib_colum].values.reshape(-1, 1)

    model = LinearRegression()
    model.fit(X, Y)

    data['Y_pred'] = model.predict(X)
    data['residual' + contrib_colum] = data[contrib_colum] - data['Y_pred']
    data['abs_residual' + contrib_colum] = data['residual' + contrib_colum].abs()
    return data


In [ ]:
data_LCAP = pd.read_csv('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/PIXIE/Complete_Data.csv')
columns_of_interest = ['delta_sum_Atom Attributions', 
                       'delta_sum_RDKit Attributions',
                       'delta_sum_fragment_contributions_Atom Attributions', 
                       'delta_sum_fragment_contributions_RDKit Attributions']
for i in columns_of_interest:
    get_residuals(data_LCAP, i)

data_LCAP.to_csv('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/PIXIE/Complete_Data_correlation_contrib.csv', index=False)

In [ ]:
data_logP = pd.read_csv('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/Lipophilicity/Complete_Data.csv')
columns_of_interest = ['delta_sum_Atom Attributions', 
                       'delta_sum_RDKit Attributions',
                       'delta_sum_SHAP Attributions',
                       'delta_sum_fragment_contributions_Atom Attributions', 
                       'delta_sum_fragment_contributions_RDKit Attributions',
                       'delta_sum_fragment_contributions_SHAP Attributions']
for i in columns_of_interest:
    get_residuals(data_logP, i)

data_logP.to_csv('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/Lipophilicity/Complete_Data_correlation_contrib.csv', index=False)

In [ ]:
data_sol = pd.read_csv('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/ESOLmoleculeNET/Complete_Data.csv')
columns_of_interest = ['delta_sum_Atom Attributions', 
                       'delta_sum_RDKit Attributions',
                       'delta_sum_fragment_contributions_Atom Attributions', 
                       'delta_sum_fragment_contributions_RDKit Attributions']
for i in columns_of_interest:
    get_residuals(data_sol, i)

data_sol.to_csv('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/ESOLmoleculeNET/Complete_Data_correlation_contrib.csv', index=False)

In [ ]:
data_TRPV1 = pd.read_csv('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/TRPV1/Complete_Data.csv')
columns_of_interest = ['delta_sum_Atom Attributions', 
                       'delta_sum_RDKit Attributions',
                       'delta_sum_SHAP Attributions',
                       'delta_sum_fragment_contributions_Atom Attributions', 
                       'delta_sum_fragment_contributions_RDKit Attributions',
                       'delta_sum_fragment_contributions_SHAP Attributions']
for i in columns_of_interest:
    get_residuals(data_TRPV1, i)

data_TRPV1.to_csv('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/TRPV1/Complete_Data_correlation_contrib.csv', index=False)

#### Confusion Matrix and Accuracy

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

def do_confusion_matrix(data, contrib_colum, dir):
    y_true = (data['delta_prediction'] > 0).astype(int)
    y_pred = (data[contrib_colum] > 0).astype(int)

    cm = confusion_matrix(y_true, y_pred)

    accuracy = accuracy_score(y_true, y_pred)
    print(contrib_colum + f" Accuracy: {accuracy:.2f}")

    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Attr Negative', 'Attr Positive'],
            yticklabels=['Pred Negative', 'Pred Positive'])
    plt.xlabel('Attributions')
    plt.ylabel('Predictions')
    plt.title(contrib_colum)
    plt.savefig(dir + contrib_colum + '_confusion_matrix.png')
    plt.show()

data_AMES = pd.read_csv('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/AMESwinter/Complete_Data.csv')
columns_of_interest = ['delta_sum_Atom Attributions', 
                       'delta_sum_RDKit Attributions',
                       'delta_sum_fragment_contributions_Atom Attributions', 
                       'delta_sum_fragment_contributions_RDKit Attributions']
for i in columns_of_interest:
    do_confusion_matrix(data_AMES, i, '/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/AMESwinter/')

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

def do_confusion_matrix(data, contrib_colum, dir):
    y_true = (data['delta_prediction'] > 0).astype(int)
    y_pred = (data[contrib_colum] > 0).astype(int)

    cm = confusion_matrix(y_true, y_pred)

    accuracy = accuracy_score(y_true, y_pred)
    print(contrib_colum + f" Accuracy: {accuracy:.2f}")

    '''
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Pred Negative', 'Pred Positive'],
            yticklabels=['Actual Negative', 'Actual Positive'])
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title(contrib_colum)
    #plt.savefig(dir + contrib_colum + '_confusion_matrix.png')
    plt.show()
    '''

In [ ]:
data_LCAP = pd.read_csv('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/PIXIE/Complete_Data.csv')
columns_of_interest = ['delta_sum_Atom Attributions', 
                       'delta_sum_RDKit Attributions',
                       'delta_sum_fragment_contributions_Atom Attributions', 
                       'delta_sum_fragment_contributions_RDKit Attributions']
for i in columns_of_interest:
    do_confusion_matrix(data_LCAP, i, '/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/PIXIE/')

In [ ]:
data_logP = pd.read_csv('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/Lipophilicity/Complete_Data.csv')
columns_of_interest = ['delta_sum_Atom Attributions', 
                       'delta_sum_RDKit Attributions',
                       'delta_sum_SHAP Attributions',
                       'delta_sum_fragment_contributions_Atom Attributions', 
                       'delta_sum_fragment_contributions_RDKit Attributions',
                       'delta_sum_fragment_contributions_SHAP Attributions']
for i in columns_of_interest:
    do_confusion_matrix(data_logP, i, '/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/Lipophilicity/')

In [ ]:
data_sol = pd.read_csv('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/ESOLmoleculeNET/Complete_Data.csv')
columns_of_interest = ['delta_sum_Atom Attributions', 
                       'delta_sum_RDKit Attributions',
                       'delta_sum_fragment_contributions_Atom Attributions', 
                       'delta_sum_fragment_contributions_RDKit Attributions']
for i in columns_of_interest:
    do_confusion_matrix(data_sol, i, '/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/ESOLmoleculeNET/')

In [ ]:
data_TRPV1 = pd.read_csv('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/TRPV1/Complete_Data.csv')
columns_of_interest = ['delta_sum_Atom Attributions', 
                       'delta_sum_RDKit Attributions',
                       'delta_sum_SHAP Attributions',
                       'delta_sum_fragment_contributions_Atom Attributions', 
                       'delta_sum_fragment_contributions_RDKit Attributions',
                       'delta_sum_fragment_contributions_SHAP Attributions']
for i in columns_of_interest:
    do_confusion_matrix(data_TRPV1, i, '/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/TRPV1/')

In [ ]:
from sklearn.metrics import confusion_matrix

y_true = (data['delta_prediction'] > 0).astype(int)
y_pred = (data['delta_sum_fragment_contributions_Atom Attributions'] > 0).astype(int)

cm = confusion_matrix(y_true, y_pred)
print(cm)

#[[TN, FP],
# [FN, TP]]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Pred Negative', 'Pred Positive'],
            yticklabels=['Actual Negative', 'Actual Positive'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

## Analyze Error on Constant Part

In [ ]:
#load data
data_attributions = pd.read_csv('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/FactorXa-Bail2023/Complete_Data.csv')

In [ ]:
import ast

data_attributions['unmatched_atom_index_1'] = data_attributions['unmatched_atom_index_1'].apply(ast.literal_eval)
data_attributions['unmatched_atom_index_2'] = data_attributions['unmatched_atom_index_2'].apply(ast.literal_eval)
data_attributions['RDKit Attributions_1'] = data_attributions['RDKit Attributions_1'].apply(ast.literal_eval)
data_attributions['RDKit Attributions_2'] = data_attributions['RDKit Attributions_2'].apply(ast.literal_eval)
data_attributions['RDKit Attributions_1_fix'] = data_attributions['RDKit Attributions_1_fix'].apply(ast.literal_eval)
data_attributions['RDKit Attributions_2_fix'] = data_attributions['RDKit Attributions_2_fix'].apply(ast.literal_eval)
#data_attributions['Atom Attributions_1'] = data_attributions['Atom Attributions_1'].apply(ast.literal_eval)
#data_attributions['Atom Attributions_2'] = data_attributions['Atom Attributions_2'].apply(ast.literal_eval)
data_attributions['Atom Attributions_1_fix'] = data_attributions['Atom Attributions_1_fix'].apply(ast.literal_eval)
data_attributions['Atom Attributions_2_fix'] = data_attributions['Atom Attributions_2_fix'].apply(ast.literal_eval)

In [ ]:
data_attributions['const_indices_1'] = data_attributions.apply(lambda row: get_unselected_atom_indices(row['smiles_1'], row['unmatched_atom_index_1']), axis=1)
data_attributions['const_indices_2'] = data_attributions.apply(lambda row: get_unselected_atom_indices(row['smiles_2'], row['unmatched_atom_index_2']), axis=1)

In [ ]:
r2_const, data_attributions = get_r2_and_summed_data_attributions_const(data_attributions, 'predictions_1', 'predictions_2', 'RDKit Attributions_1', 'RDKit Attributions_2', "RDKit_Attributions", 'const_indices_1' , 'const_indices_2')

plot_2D(['$r^2$(RDKit) = ' + str(f"{r2_const:.2f}")], 'upper left', data_attributions['delta_prediction'], data_attributions['delta_sum_const_contributions_RDKit_Attributions'],
        '$\Delta$Predictions MMP', '$\Delta$Attributions MMP (Constant)', '/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/FactorXa-Bail2023/PREDvsCONTRIBUTIONSconstRDKITattributor.png', 
        '#758ECD',
        include_line=True, line_style='None')

r2_const, data_attributions = get_r2_and_summed_data_attributions_const(data_attributions, 'predictions_1', 'predictions_2', 'Atom Attributions_1', 'Atom Attributions_2', "Atom_Attributions", 'const_indices_1' , 'const_indices_2')

plot_2D(['$r^2$(XSMILES) = ' + str(f"{r2_const:.2f}")], 'upper left', data_attributions['delta_prediction'], data_attributions['delta_sum_const_contributions_Atom_Attributions'],
        '$\Delta$Predictions MMP', '$\Delta$Attributions MMP (Constant)', '/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/FactorXa-Bail2023/PREDvsCONTRIBUTIONSconstATOMattributor.png', 
        '#10384f',
        include_line=True, line_style='None')

In [ ]:
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.colors as mcolors

def plot_histogram_one_dataset(data, colum_of_interest, label, color, attr_method, save_dir):

    #get standard deviation
    std_dev = np.std(data[colum_of_interest])

    # color
    base_color = mcolors.to_rgba(color) 
    color_with_alpha = base_color[:3] + (0.5,)

    # Plotting
    plt.figure(figsize=(10, 6))
    plt.hist(data[colum_of_interest], bins=30, color=color_with_alpha, edgecolor=color, label=f"{attr_method} \n(Standard Deviation: {std_dev:.2f})")  
    
    plt.rcParams['font.family'] = 'arial'
    plt.xlabel(label, fontsize=18)
    plt.ylabel('MMP Count', fontsize=18)
    plt.legend(fontsize=16, loc='upper left')


    # Save plot
    filename = f"{attr_method.replace(' ', '_').lower()}_histogram.png"
    save_path = os.path.join(save_dir, filename)
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

    print(f"Histogram saved to {save_path}")

In [ ]:
plot_const_histogram(data_attributions, 'RDKit_Attributions', '#758ECD', '/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/FactorXa-Bail2023/')

## MorganGenerator

In [ ]:
def weights_morgan(smiles, coefficients_total):
    mol = Chem.MolFromSmiles(smiles, sanitize=False)
    Chem.SanitizeMol(mol)#to keep the explicit hydrogens

    bit_info = {}
    #fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048, bitInfo=bit_info)
    generator = GetMorganGenerator(radius=2, fpSize=2048)
    fp = generator.GetFingerprint(mol, bitInfo=bit_info)

    #print(bit_info)

    atom_weights = calculate_atom_weights(mol, coefficients_total, bit_info)

    return atom_weights

def get_SHAP_Morgan_attributions(data, feature_column, smiles_column, model, explainer):
    prep_data = get_features(data, [feature_column])

    shap_values = explainer.shap_values(model.named_steps['scaler'].transform(prep_data))
    print("Shap values are calculated.")

    atom_weights_list = []

    for nr, (index, row) in enumerate(data.iterrows(), 1):
        smiles = row[smiles_column]
        shap_nr = nr - 1
        atom_weights = weights_morgan(smiles, shap_values[shap_nr])
        atom_weights_values = list(atom_weights.values())
        atom_weights_list.append(atom_weights_values)

    data['SHAP Attributions'] = atom_weights_list

    return data

def get_morgan_fingerprint(smiles):
    """
    Calculates the Morgan Fingerprint (2028 bits, radius of 2) for the input smiles.

    Keyword arguments:
    -- smiles: Smiles for the which the fingerprint should be calculated.

    Returns:
    -- fingerprint: Morgan Fingerprint as array to the respective smiles.
    """
    mol = Chem.MolFromSmiles(smiles, sanitize=False)
    Chem.SanitizeMol(mol)#to keep the explicit hydrogens
    
    if mol is not None:
        generator = GetMorganGenerator(radius=2, fpSize=2048)
        fingerprint = generator.GetFingerprint(mol)
        #fingerprint = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048)
        fingerprint = fingerprint.ToBitString()
        fingerprint = np.array(list(fingerprint))
        return fingerprint
    else:
        return None

In [ ]:
#load data
data_attributions = pd.read_csv('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/FactorXa-Bail2023/Complete_Data.csv')
data_attributions['Morgan_Fingerprint 2048Bit 2rad'] = data_attributions['smiles_1'].apply(get_morgan_fingerprint)
model = pickle.load(open('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/FactorXa-Bail2023/' + "model.pkl", 'rb'))
explainer = shap.TreeExplainer(model.named_steps['model'])
get_SHAP_Morgan_attributions(data_attributions, 'Morgan_Fingerprint 2048Bit 2rad', 'smiles_1', model, explainer)

# Calculate Crippen

In [ ]:
data_logP = pd.read_csv('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/Crippen-Rassmussen/ZINC_250k.smi', header=None)

In [ ]:
data_logP.columns = ['smiles']  # Change 'YourColumnName' to whatever you prefer

# Add a column with row numbers starting from 1
data_logP.insert(0, 'No', range(1, len(data_logP) + 1))

In [ ]:
data_logP

In [ ]:
from rdkit.Chem import Crippen

def calculate_logp(smiles):
    mol = Chem.MolFromSmiles(smiles, sanitize=False)
    Chem.SanitizeMol(mol)
    if mol:
        return Crippen.MolLogP(mol)
    return None

data_logP["crippen_logP"] = data_logP["smiles"].apply(calculate_logp)

In [ ]:
data_logP.to_csv('/Users/kerrinjanssen/Nextcloud/PhD/Bayer/XAI/Crippen-Rassmussen/ZINC-Crippen.csv', index=False)